In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')
%cd "drive/My Drive/shap-dim-reduction/shap-dim-reduction"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/shap-dim-reduction/shap-dim-reduction


In [2]:
try:
  ! git config --global user.email "EMAIL"
  ! git config --global user.name "USERNAME"
  ! git pull
except:
  ! git stash 
  ! git pull

remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 8 (delta 6), reused 5 (delta 3), pack-reused 0
Unpacking objects: 100% (8/8), done.
From https://github.com/parkervg/shap-dim-reduction
   27de20c..bbab4c9  master     -> origin/master
Updating 27de20c..bbab4c9
Fast-forward
 lib/ProcessEmbeddings.py | 8 +++++++-
 1 file changed, 7 insertions(+), 1 deletion(-)


In [3]:
try:
  import shap 
except:
  !pip install shap

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
import bz2
import re
from nltk.tokenize import word_tokenize
import numpy as np
from sklearn.linear_model import LogisticRegression
from lib.ProcessEmbeddings import WordEmbeddings
from sklearn.utils import shuffle

In [6]:
def standard_glove():
  WE = WordEmbeddings(vector_file="embeds/glove.6B.300d.txt")
  word_vectors = WE.get_vector_dict()
  wvec_dim = WE.embeds.shape[1]
  return word_vectors, wvec_dim

In [7]:
def shap_glove(task):
  WE = WordEmbeddings(vector_file="embeds/glove.6B.300d.txt")
  dims = WE.shap_dim_reduction(task=task, k=50)
  word_vectors = WE.get_vector_dict()
  wvec_dim = WE.embeds.shape[1]
  return word_vectors, wvec_dim, dims

In [8]:
def rand_dims_glove():
  WE = WordEmbeddings(vector_file="embeds/glove.6B.300d.txt")
  dims = WE.rand_dim_reduction(k=50)
  word_vectors = WE.get_vector_dict()
  wvec_dim = WE.embeds.shape[1]
  return word_vectors, wvec_dim, dims

In [9]:
def labels_text(x):
  label=[]
  text=[]
  for line in bz2.BZ2File(x):
    decode = line.decode("utf-8")
    label.append(int(decode[9]) - 1)
    text.append(decode[10:].strip())
  return np.array(label),text

train_label, train_text = labels_text('data/amazon-reviews/train.ft.txt.bz2')
test_label, test_text = labels_text('data/amazon-reviews/test.ft.txt.bz2')

We're only using a subsection of the dataset, so we shuffle it.

In [10]:
train_text, train_label = shuffle(train_text, train_label)
test_text, test_label = shuffle(test_text, test_label)

In [11]:
train_text = train_text[:10000]
train_label = train_label[:10000]
test_text = test_text[:5000]
test_label = test_label[:5000]

Make sure there's no overlap in amazon-review data and CR data

In [29]:
# all_lines = []
# with open("SentEval/data/downstream/CR/custrev.neg", "r") as f:
#   for line in f:
#     all_lines.append(line.strip())
# with open("SentEval/data/downstream/CR/custrev.pos", "r") as f:
#   for line in f:
#     all_lines.append(line.strip())
# for line1 in all_lines:
#   for line2 in train_text + test_text:
#     if re.sub(r'\s+', '', line1.lower()) == re.sub(r'\s+', '', line2.lower()):
#       print("DUPLICATE")

In [12]:
def get_sent_vectors(sents, word_vectors, wvec_dim):
  """
  Averages over all word vectors in a sentence.
  """
  embeddings = []
  for sent in sents:
      sentvec = []
      for word in word_tokenize(sent):
        try:
            sentvec.append(word_vectors[word.lower()])
        except:
            pass
      if not sentvec:
          vec = np.zeros(wvec_dim)
          sentvec.append(vec)
      sentvec = np.mean(sentvec, 0)
      embeddings.append(sentvec)
  embeddings = np.vstack(embeddings)
  return embeddings

In [13]:
word_vectors, wvec_dim = standard_glove()
Xtrain_glove = get_sent_vectors(train_text, word_vectors, wvec_dim)
Xtest_glove = get_sent_vectors(test_text, word_vectors, wvec_dim)

word_vectors, wvec_dim, cr_dims = shap_glove("CR")
Xtrain_shap_cr = get_sent_vectors(train_text, word_vectors, wvec_dim)
Xtest_shap_cr = get_sent_vectors(test_text, word_vectors, wvec_dim)

word_vectors, wvec_dim, trec_dims = shap_glove("TREC")
Xtrain_shap_trec = get_sent_vectors(train_text, word_vectors, wvec_dim)
Xtest_shap_trec = get_sent_vectors(test_text, word_vectors, wvec_dim)

word_vectors, wvec_dim, mr_dims = shap_glove("MR")
Xtrain_shap_mr = get_sent_vectors(train_text, word_vectors, wvec_dim)
Xtest_shap_mr = get_sent_vectors(test_text, word_vectors, wvec_dim)

word_vectors, wvec_dim, rand_dims = rand_dims_glove()
Xtrain_rand = get_sent_vectors(train_text, word_vectors, wvec_dim)
Xtest_rand = get_sent_vectors(test_text, word_vectors, wvec_dim)

2020-11-08 19:13:17 INFO     Loading vectors at embeds/glove.6B.300d.txt...
2020-11-08 19:13:49 INFO     Loading vectors at embeds/glove.6B.300d.txt...
2020-11-08 19:14:08 INFO     Found 5282 words with word vectors, out of             5677 words
2020-11-08 19:14:08 INFO     Generating sentence embeddings
2020-11-08 19:14:08 INFO     Generated sentence embeddings
2020-11-08 19:14:08 INFO     Training sklearn-LogReg with (inner) 5-fold cross-validation
2020-11-08 19:14:11 INFO     Best param found at split 1: l2reg = 2                 with score 78.01
2020-11-08 19:14:13 INFO     Best param found at split 2: l2reg = 2                 with score 77.45
2020-11-08 19:14:16 INFO     Best param found at split 3: l2reg = 4                 with score 78.71
2020-11-08 19:14:18 INFO     Best param found at split 4: l2reg = 4                 with score 78.34
2020-11-08 19:14:21 INFO     Best param found at split 5: l2reg = 1                 with score 77.65
2020-11-08 19:14:21 INFO     Original a

Full 300 dimension Glove accuracy

In [14]:
clf = LogisticRegression()
clf.fit(Xtrain_glove, train_label)
print(round(clf.score(Xtest_glove, test_label), 2))

0.82


In [16]:
WE = WordEmbeddings(vector_file="embeds/glove.6B.300d.txt")
glove_dims = WE.top_shap_dimensions(clf, np.vstack((Xtrain_glove, Xtest_glove)), k=50)
Xtrain_glove_reduced = np.take(Xtrain_glove, indices=glove_dims, axis=1)
Xtest_glove_reduced = np.take(Xtest_glove, indices=glove_dims, axis=1)

2020-11-08 19:18:00 INFO     Loading vectors at embeds/glove.6B.300d.txt...
2020-11-08 19:18:20 INFO     Classifier has 2 classes


SHAP amazon-review reduced 50 dimension Glove accuracy

In [17]:
clf = LogisticRegression()
clf.fit(Xtrain_glove_reduced, train_label)
print(round(clf.score(Xtest_glove_reduced, test_label), 2))

0.8


SHAP CR reduced 50 dimension accuracy



In [18]:
clf = LogisticRegression()
clf.fit(Xtrain_shap_cr, train_label)
print(round(clf.score(Xtest_shap_cr, test_label), 2))

0.78


SHAP reduced 50 dimensions on TREC

In [19]:
clf = LogisticRegression()
clf.fit(Xtrain_shap_trec, train_label)
print(round(clf.score(Xtest_shap_trec, test_label), 2))

0.76


SHAP reduced 50 dimensions on MR

In [20]:
clf = LogisticRegression()
clf.fit(Xtrain_shap_mr, train_label)
print(round(clf.score(Xtest_shap_mr, test_label), 2))

0.77


Randomly reduced 50 dimension accuracy

In [21]:
clf = LogisticRegression()
clf.fit(Xtrain_rand, train_label)
print(round(clf.score(Xtest_rand, test_label), 2))

0.74


In [24]:
def intersect(a, b):
  return len(list(set(a) & set(b)))
print(f"Intersecting dims between CR and amazon-review: {intersect(glove_dims, cr_dims)}")
print()
print(f"Intersecting dims between MR and amazon-review: {intersect(glove_dims, mr_dims)}")
print()
print(f"Intersecting dims between TREC and amazon-review: {intersect(glove_dims, trec_dims)}")
print()
print(f"Intersecting dims between random dims and amazon-review: {intersect(rand_dims, trec_dims)}")

Intersecting dims between CR and amazon-review: 19

Intersecting dims between MR and amazon-review: 17

Intersecting dims between TREC and amazon-review: 11

Intersecting dims between random dims and amazon-review: 11
